## Расчет продуктовых метрик. 3 кейс курса Аналитик данных

В этом кейсе необходимо рассчитать метрики:

- retention
- rolling retention
- lifetime
- churn rate
- mau
- wau

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/a-antonchev/7850fe93344af6ba431219e820f6647a/raw/registrations.csv
!wget https://gist.github.com/a-antonchev/93388a128117ab6ba85225b9866f2e35/raw/entries.csv

Файлами для работы являются `registrations.csv` и `entries.csv`.\
В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### 0. Предобработка данных

In [1]:
from datetime import datetime, date

In [2]:
def get_data(file):
    """
    Считать данные из входного файла и преобразовать в список
    Parameters:
    -----------
    file : {file}	
	       Файл формата CSV
    Returns:
    --------
    result : {list}
             Список списков, каждый элемент представляет собой пару [id пользователя, дата]
    """
    with open(file, 'r') as f:
        f.readline() # считать первую строку с заголовками столбцов, она не требуется
        result = f.read().split('\n')
    # разделить каждую строку по разделителю ';'
    result = [result[i].split(';') for i in range(len(result))]
    # преобразовать строковую дату в date (можно оставить и datetime)
    result = [[result[i][0], datetime.strptime(result[i][1], '%Y-%m-%d').date()] for i in range(len(result))]
    return result

In [ ]:
registrations = get_data('registrations.csv') # создать список регистраций пользователей
entries = get_data('entries.csv') # создать список заходов пользователей
print(entries[:3])
print(registrations[:3])

In [ ]:
# Сформировать список заходов пользователей на платформу (user_id, entry_date, date_diff),\
# где date_diff - разница в днях между датой регистрации и датой захода

entries_temp = []
while entries:
    # изъять последний элемент (user_id, entry_date)
    last = entries.pop()
    # найти user_id
    used_id = last[0]
    for r in registrations:
        # ищем в регистрациях элемент с выбранным user_id
        if r[0] == used_id:
            # рассчитать разницу в днях между датой регистрации и датой захода
            day_diff = (last[1] - r[1]).days
            # добавить разницу в днях в элемент => (user_id, entry_date, date_diff)
            last.append(day_diff)
            # преобразовать элемент в кортеж (чтобы потом 'прогнать' через set и оставить только уникальные элементы) и добавить в новый список
            entries_temp.append(tuple(last))
            break

# 'прогнать' через set, чтобы оставить только уникальные заходы
entries = list(set(entries_temp))
entries[:3]

### 1. Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе
Cохраните результат в переменную `retention_15_day`

Примечание: результат округлите до 5 знаков после запятой

In [ ]:
# расчитать количество заходов в заданный день 'day'
# отбираем user_id, у которых разница в днях между регистрацией и заходом равна 'day' и которые зарегистрировались в январе
# считаем количество (len) user_id

# количество заходов в день 0
entries_0_day = len([d[0] for d in entries if d[2] == 0 and d[1] <= date(2021, 1, 31) and d[1] >= date(2021, 1, 1)])
# количество заходов в день 15
entries_15_day = len([d[0] for d in entries if d[2] == 15 and d[1] <= date(2021, 1, 31) and d[1] >= date(2021, 1, 1)])

retention_15_day = round(entries_15_day * 100.0 / entries_0_day, 5)

print(f'Retention 15 дня: {retention_15_day}')

### 2. Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты
Сохраните результат в переменную `rolling_retention`

Примечание: результат округлите до 5 знаков после запятой

In [ ]:
# количество заходов в день 0 или позже (применить отбор уникальных user_id через set)
entries_0_day_and_later = len({d[0] for d in entries if d[2] >= 0 and d[1] <= date(2021, 1, 31) and d[1] >= date(2021, 1, 1)})
# количество заходов в день 30 или позже
entries_30_day_and_later = len({d[0] for d in entries if d[2] >= 30 and d[1] <= date(2021, 1, 31) and d[1] >= date(2021, 1, 1)})

rolling_retention = round(entries_30_day_and_later * 100.0 / entries_0_day_and_later, 5)

print(f'Rolling-retention 30 дня: {rolling_retention}')

In [ ]:
entries_30_day_and_later, entries_0_day_and_later

### 3. Посчитайте Lifetime по всем пользователям, как интеграл от n-day retention
Сохраните результат в переменную `lifetime`

Примечание: результат округлите до 5 знаков после запятой

In [ ]:
date_diff_set = {el[2] for el in entries} # отобрать уникальные 'date_diff' через преобразование в множество
users = len(registrations) # рассчитать количество пользователей, как длину списка регистраций

entries_all = 0

# пройтись по всем датам
for day in date_diff_set:
    # суммировать количество заходов в каждый 'date_diff' = интеграл по n-day retention
    entries_all += len({d[0] for d in entries if d[2] == day})

lifetime = round(entries_all / users, 5)

print(f'Lifetime: {lifetime}')

### 4. Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям
Сохраните результат в переменную `churn_29`

In [ ]:
# отобрать уникальные user_id (через set), у которых разница в днях между регистрацией и заходом больше или равна 'day'
# то есть, эти пользователи заходили в 29 день и после = оставшиеся пользователи
user_29 = len({d[0] for d in entries if d[2] >= 29})

churn_29 = (users - user_29) / users

print(f'Churn rate 29 дня: {churn_29}')

### 5. Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей
Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

Примечание: последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [ ]:
dec_mau = len({el[0] for el in entries if el[1].year == 2021 and el[1].month == 12})
dec_wau = len({el[0] for el in entries if el[1] >= date(2021, 12, 25) and el[1] <= date(2021, 12, 31)})
dec_dau = len({el[0] for el in entries if el[1] == date(2021, 12, 31)})

print(f'MAU в декабре: {dec_mau}')
print(f'WAU в декабре: {dec_wau}')
print(f'DAU в декабре: {dec_dau}')

### 5. Посчитайте Mau, Wau, Dau усредненные
Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

Примечание: результаты округлите до 5 знаков после запятой

In [ ]:
years = {el[1].year for el in entries} # получить множество всех годов из дат entries
months = {el[1].month for el in entries} # получить множество всех месяцев из дат entries
pre_mau = 0
# для каждого года и месяца расчитать количество уникальных пользователей накопительным итогом
for year in years:
    for month in months:
        pre_mau += len({el[0] for el in entries if el[1].year == year and el[1].month == month})

avg_mau = round(pre_mau / 12, 5)

print(f'MAU усредненный: {avg_mau}')

In [ ]:
# avg_wau вариант 1, считаем номера недель по ISO 8601

pre_wau = 0
for i in range(1, 54):
    pre_wau += len({el[0] for el in entries if el[1].isocalendar().week == i})

avg_wau = round(pre_wau / 53, 5) # здесь 53 части

print(f'WAU усредненный: {avg_wau}')

In [ ]:
dates = {el[1] for el in entries} # получить множество всех дат из дат entries
pre_dau = 0
for date in dates:
    # для каждой даты расчитать количество уникальных пользователей накопительным итогом
    pre_dau += len({el[0] for el in entries if el[1] == date})

avg_dau = round(pre_dau / len(dates), 5)
avg_wau = round(pre_wau / 53, 5) # здесь 53 части

print(f'DAU усредненный: {avg_dau}')